In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

C:\Users\Kyujin\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Kyujin\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Kyujin\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
mnist=input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [4]:
x=tf.placeholder(tf.float32, [None, 28,28,1])
y=tf.placeholder(tf.float32, [None, 10])
keep_prob=tf.placeholder(tf.float32)


In [31]:
w1=tf.Variable(tf.random_normal([3,3,1,32]))
#[3,3]: 필터(커널) 크기, 1:채널 특성, 32: 필터개수
#strides=[1, 칸, 줄, 1]
L1=tf.nn.conv2d(x,w1,strides=[1,1,1,1], padding='SAME')
L1=tf.nn.relu(L1)
L1=tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

w2=tf.Variable(tf.random_normal([3,3,32,64]))
#[3,3]: 필터(커널) 크기, 32:채널 특성, 64: 필터개수
#strides=[1, 칸, 줄, 1]
L2=tf.nn.conv2d(L1,w2,strides=[1,1,1,1], padding='SAME')
L2=tf.nn.relu(L2)
L2=tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
#L2 conv => 

#FC 계층: 입력 7*7*64 -> 출력 256
w3=tf.Variable(tf.random_normal([7*7*64, 256]))
#[3,3]: 필터(커널) 크기, 32:채널 특성, 64: 필터개수
#strides=[1, 칸, 줄, 1]
L3=tf.reshape(L2,[-1,7*7*64])
L3=tf.matmul(L3,w3)
L3=tf.nn.relu(L3)
L3=tf.nn.dropout(L3, keep_prob)

w4=tf.Variable(tf.random_normal([256,10]))
model=tf.matmul(L3,w4)

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y))
optimizer=tf.train.AdamOptimizer(0.01).minimize(cost)

In [35]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())
batch_size=100
total_batch=int(mnist.train.num_examples/batch_size)
for epoch in range(15):
    total_cost=0
    for i in range(total_batch):
            batch_xs, batch_ys=mnist.train.next_batch(batch_size)
            batch_xs= batch_xs.reshape(-1,28,28,1)
            _,cv= sess.run([optimizer, cost], feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.7})
            total_cost+=cv
    print("Epoch:", "%d" %(epoch+1),
         "평균 cost: ","{:.3f}".format(total_cost/total_batch))
print("모델 생성")

is_correct=tf.equal(tf.argmax(model, 1), tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(is_correct,tf.float32))
print("정확도:", sess.run(accuracy, feed_dict={x:mnist.test.images.reshape(-1,28,28,1),
                                            y:mnist.test.labels,
                                           keep_prob:1}))

정확도: 0.3254


In [36]:
x=tf.placeholder(tf.float32, [None, 28,28,1])
y=tf.placeholder(tf.float32, [None, 10])

In [37]:
L1=tf.layers.conv2d(x, 32, [3,3], activation=tf.nn.relu)
L1=tf.layers.max_pooling2d(L1,[2,2],[2,2])
L1=tf.layers.dropout(L1,0.7)

L1=tf.layers.conv2d(L2, 64, [3,3], activation=tf.nn.relu)
L1=tf.layers.max_pooling2d(L2,[2,2],[2,2])
L1=tf.layers.dropout(L2,0.7)

L1=tf.contrib.layers.flatten(L2)
L1=tf.layers.dense(L3,256,activation=tf.nn.relu)
L1=tf.layers.dropout(L3,0.5)

model=tf.layers.dense(L3, 10, activation=None)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y))
train=tf.train.AdamOptimizer(0.001).minimize(cost)

W0819 14:54:49.792642 11060 deprecation.py:323] From <ipython-input-37-b0d7a4d36340>:1: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0819 14:54:49.843509 11060 deprecation.py:506] From C:\Users\Kyujin\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 14:54:50.289316 11060 deprecation.py:323] From <ipython-input-37-b0d7a4d36340>:2: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0819 14:54:50.580542 11060 deprecation.py:323] From <ipython-input-37-b0d7a4d36340>

In [38]:
from keras import layers
from keras import models

Using TensorFlow backend.


In [51]:
model=models.Sequential()
model.add(layers.Conv2D(32, (3,3),activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64, (3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64, (3,3),activation='relu'))
model.add(layers.Flatten())

In [52]:
model.add(layers.Dense(64, activation='relu'))

In [53]:
model.add(layers.Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                36928     
__________

In [47]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [48]:
(train_images, train_labels), (test_images, test_labels)= mnist.load_data()

In [49]:
train_images=train_images.reshape((60000, 28,28, 1))
train_images=train_images.astype('float32')/255
test_images=test_images.reshape((10000, 28,28, 1))
test_images=test_images.astype('float32')/255

In [54]:
train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)

In [55]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

W0819 15:06:30.511439 11060 deprecation_wrapper.py:119] From C:\Users\Kyujin\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [57]:
model.fit(train_images, train_labels, epochs=5, batch_size=100)

W0819 15:07:59.716256 11060 deprecation.py:323] From C:\Users\Kyujin\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
60000/60000 [==============================] - 70s 1ms/step - loss: 0.2156 - acc: 0.9313
Epoch 2/5
60000/60000 [==============================] - 66s 1ms/step - loss: 0.0542 - acc: 0.9830
Epoch 3/5
60000/60000 [==============================] - 68s 1ms/step - loss: 0.0364 - acc: 0.9889
Epoch 4/5
60000/60000 [==============================] - 72s 1ms/step - loss: 0.0276 - acc: 0.9912
Epoch 5/5
60000/60000 [==============================] - 89s 1ms/step - loss: 0.0209 - acc: 0.9934


In [58]:
test_loss,test_acc=model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 3s 274us/step


In [59]:
test_acc

0.988

In [69]:
import os
import shutil
# 원본 데이터셋을 압축 해제한 디렉터리 경로
original_dataset_dir = 'dogs-vs-cats/train/train'

# 소규모 데이터셋을 저장할 디렉터리
base_dir = 'dogs-vs-cats/cats_and_dogs_small'
if os.path.exists(base_dir):  # 반복적인 실행을 위해 디렉토리를 삭제합니다.
    shutil.rmtree(base_dir)   # 이 코드는 책에 포함되어 있지 않습니다.
os.mkdir(base_dir)

# 훈련, 검증, 테스트 분할을 위한 디렉터리
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 훈련용 고양이 사진 디렉터리
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# 훈련용 강아지 사진 디렉터리
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# 검증용 고양이 사진 디렉터리
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# 검증용 강아지 사진 디렉터리
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# 테스트용 고양이 사진 디렉터리
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# 테스트용 강아지 사진 디렉터리
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

# 처음 1,000개의 고양이 이미지를 train_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# 다음 500개 고양이 이미지를 validation_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 고양이 이미지를 test_cats_dir에 복사합니다
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# 처음 1,000개의 강아지 이미지를 train_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 강아지 이미지를 validation_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# 다음 500개 강아지 이미지를 test_dogs_dir에 복사합니다
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)


In [75]:
train_cats_dir

'dogs-vs-cats/cats_and_dogs_small\\train\\cats'

In [76]:
model = Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(Conv2D(64,(3,3), activation = 'relu', input_shape = X.shape[1:]))
model.add(MaxPooling2D(pool_size = (2,2)))
# Add another:
model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])


NameError: name 'Sequential' is not defined